Измалкова Дарья БКЛ182

# Домашнее задание №1

In [1]:
import pandas as pd

import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk import tokenize
import math
from operator import itemgetter


from summa import keywords
import RAKE

from collections import Counter

## Данные

Собрала корпус из 16 аннотаций и ключевых слов к статьям с ресурса researchgate.net (статьи выбирала рандомные), ссылки к конкретным статьям в столбце "link".

In [2]:
data = pd.read_csv('nlp_hw1_corpus.csv')

In [3]:
data

,abstract,source_keywords,my_keywords,link
0,Social scientists are increasingly interested ...,"Environmental values, Animal well-being, Farm ...","farm animals, animal well-being, environmental...",https://www.researchgate.net/publication/35044...
1,The main aim of this article is to start a dis...,"Computational social science, Cultural pattern...","social pattern, pattern, sociology",https://www.researchgate.net/publication/35520...
2,Integrating the dynamics and interconnections ...,"Asian Drylands Belt, social‐environmental syst...","social-environmental system, SES, Asian Drylan...",https://www.researchgate.net/publication/35518...
3,Citizen science research has been rapidly expa...,"citizen science, political engagement, educati...","youth, citizen science, science efficacy, poli...",https://www.researchgate.net/publication/35519...
4,This research aims to investigate the potentia...,"Green advertisement, Consumer empowerment, Cus...","consumer empowerment, organic food, green ads,...",https://www.researchgate.net/publication/35505...
5,Recognition of others' identity through facial...,"person perception, face processing, semantic m...","perception, person knowledge, facial features,...",https://www.researchgate.net/publication/35419...
6,Evidence suggests parents of children who expe...,"Post-traumatic stress disorder, Prevalence, Ri...","post-traumatic stress disorder, PTSD, parents,...",https://www.researchgate.net/publication/35480...
7,Current theories of artificial intelligence (A...,"emotion, thinking, emotional thinking, AI cons...","artificial intelligence, ai, intelligence, emo...",https://www.researchgate.net/publication/35567...
8,When we say that what two people feel for each...,"true love, ordinary concept, prototype, goodne...","true love, love",https://www.researchgate.net/publication/34869...
9,"During the last decade, intellectual humility ...","Intellectual humility, Measurement, Intellectu...","intellectual humility, definition, measurement",https://www.researchgate.net/publication/35488...


Решила в качестве эталона использовать объединение оригинальных и моих ключевых слов. Оригинальные ключевые слова относятся к стаьям в целом, поэтому иногда там встречаются слова/словосочетания, которые не встречаются (или не частно встречаются) в аннотации. С другой стороны, я не разбираюсь в темах некоторых статей и не уверена, что могу правильно оценить какие слова ключевые, а какие - нет. Также, в оригинальных иногда отсутствуют абревиатуры (есть только термин целиком), которые часто используются в тексте.

In [4]:
def keyword_union(source_keys, my_keys):
    source_set = set(source_keys.lower().split(', '))
    my_set = set(my_keys.lower().split(', '))
    return ', '.join(list(source_set | my_set))

In [5]:
data['keywords'] = data.apply(lambda x: keyword_union(x.source_keywords,
                                                              x.my_keywords), axis=1)

## Извлечение ключевых слов

### Rake

Лемматизирую данные ключевые слова и тексты.

In [6]:
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

def lemmatize(text):
    tokens = nlp(text)
    lemmas = ' '.join([word.lemma_ for word in tokens
                      if word.lemma_ not in stop_words and word.lemma_ != '-PRON-'])
    return lemmas

In [7]:
data['keywords_lem'] = data['keywords'].apply(lemmatize)
data['abstract_lem'] = data['abstract'].apply(lemmatize)

In [8]:
data['keywords_lem'] = data['keywords_lem'].apply(lambda x: x.split(' , '))

### Rake

In [9]:
stop = 'SmartStoplist.txt'
rake = RAKE.Rake(stop)

Rake для некоторых лемматизированных текстов не находил ключевые слова (полагаю, что это из-за маленького размера текстов), поэтому воспользуюсь результатами и нелемматизированного текста.

In [10]:
data['rake_not_lem'] = data['abstract'].apply(lambda x: rake.run(x, maxWords=3, minFrequency=2))
data['rake_lem'] = data['abstract_lem'].apply(lambda x: rake.run(x, maxWords=3, minFrequency=2))

In [11]:
def keywords_only(a_list):
    keywords = [word[0] for word in a_list]
    return keywords

In [12]:
data['rake_lem'] = data['rake_lem'].apply(keywords_only)
data['rake_not_lem'] = data['rake_not_lem'].apply(keywords_only)

In [13]:
data['rake_lem']

0                                                    []
1                                                    []
2                                         [social, adb]
3                                    [science efficacy]
4                                                    []
5     [individual similar personality, participant, ...
6                                  [trauma factor, 0 %]
7                                                    []
8                                                    []
9                                                    []
10                [engage risk, make task, participant]
11                                        [friendly ai]
12                                                 [ai]
13                                                   []
14                                                   []
15                                                   []
Name: rake_lem, dtype: object

In [14]:
data['rake_not_lem']

0     [farm animal well-, environmental values, valu...
1     [social pattern, concept, patterns, discussion...
2        [18 political entities, metric, economic, adb]
3     [science efficacy, efficacy, interest, partici...
4                      [purchase intentions, consumers]
5     [facial features, facial identity, similar per...
6     [trauma, parents, ptsd, child, development, cl...
7     [ai consciousness, thinking, intelligence, ai,...
8                        [true love, picks, real, love]
9                    [intellectual humility, framework]
10    [decision-making task, risky options, risk lev...
11                  [ai, development, humans, question]
12    [ai, animals, technologies, application, aspec...
13    [brand connection, marketing communications, c...
14    [covid-19 pandemic, complex disease, ai techni...
15    [ci methods, fsc, problems, ci, stages, method...
Name: rake_not_lem, dtype: object

In [15]:
data['keywords_lem']

0     [animal welfare, animal agriculture, farm anim...
1     [sociology, pattern, cultural pattern, social ...
2     [asian drylands belt, social - environmental s...
3     [political engagement, political interest, cit...
4     [green advertisement, advertisement, green ad,...
5     [reverse correlation, person knowledge, face p...
6     [prevalence, post - traumatic stress disorder,...
7     [intelligence, emotional thinking, cognition, ...
8     [love, ordinary concept, prototype, realness, ...
9     [measurement, definition, wisdom, jingle - jan...
10    [risky decision making, longshot, risk, socioe...
11    [control problem, fai, superintelligence, impa...
12    [artificial intelligence, animal experiment, a...
13    [autonomous vehicle, consumer resistance, self...
14    [covid-19, electronic health record, machine l...
15    [computational intelligence, food supply chain...
Name: keywords_lem, dtype: object

### TextRank

TextRank лучше работает с лемматизированным текстом

In [16]:
data['TextRank_not_lem'] = data['abstract'].apply(lambda x: 
                                                  keywords.keywords(x, 
                                                                    language='english', 
                                                                    additional_stopwords=stopwords.words('english'), 
                                                                    scores=True))
data['TextRank_lem'] = data['abstract_lem'].apply(lambda x: 
                                                  keywords.keywords(x, 
                                                                    language='english', 
                                                                    additional_stopwords=stopwords.words('english'), 
                                                                    scores=True))

In [17]:
data['TextRank_lem'] = data['TextRank_lem'].apply(keywords_only)
data['TextRank_not_lem'] = data['TextRank_not_lem'].apply(keywords_only)

In [18]:
data['TextRank_lem']

0     [belief farm, value, social, research, importa...
1     [useful, discussion social pattern, concept, u...
2     [metric, assess, different social, product, pr...
3     [science, participation, participate, scientif...
4     [effect, effective, ad effectiveness, company,...
5     [identity facial, similarly, participant, stud...
6     [trauma, ptsd, suggest parent child, factor, d...
7     [cognitive, emotional, thinking, human emotion...
8     [true love, real, instance, people feel, exper...
9     [measure, measurement, research, intellectual ...
10    [end, ending, risk, option, motivational, moti...
11    [human, development, develop, agi, ai, ethical...
12    [animal, application, ai, technology, paper, a...
13    [study, influence brand, radical innovation, c...
14    [pandemic, disease, covid, clinical, health, r...
15    [ci, fsc, problem, stage, method, identify, da...
Name: TextRank_lem, dtype: object

In [19]:
data['TextRank_not_lem']

0     [social scientists, environmental, values, psy...
1     [main aim, article, ends, pattern, patterns, l...
2     [metric, metrics, social, environmental, produ...
3     [science, efficacy, efficacious, scientific, s...
4     [research aims, increase, people, online exper...
5     [identity, identities, facial, participant, pa...
6     [trauma, traumas, effect, effects, ptsd, estim...
7     [current theories, theory, social, anthropolog...
8           [people feel, use, distinguish, clarifying]
9     [research, decade intellectual humility, futur...
10    [risk, decision making, previous studies, work...
11    [substantial increase, numerous issues, existe...
12    [ai, research, debated, debates certain, acade...
13    [studies, study, consumer, consumers, brands, ...
14    [pandemic, clinical, health, care, disease, co...
15    [fsc, ci, problems, data, supply, complexity, ...
Name: TextRank_not_lem, dtype: object

### Tf-Idf

In [20]:
def tf_idf(doc, n=0.023):
    
    doc = clean_text(doc)
    total_words = doc.split()
    bi_grams = list(nltk.ngrams(total_words, 2))
    tri_grams = list(nltk.ngrams(total_words, 3))
    bigrams = [' '.join(gram) for gram in bi_grams]
    trigrams = [' '.join(gram) for gram in tri_grams]

    total_sentences = tokenize.sent_tokenize(doc)

# TF    
    tf_score1 = tf(total_words)
    tf_score2 = tf(bigrams)
    tf_score3 = tf(trigrams)

    idf_score1 = idf(total_words, total_sentences)
    idf_score2 = idf(bigrams, total_sentences)
    idf_score3 = idf(trigrams, total_sentences)


# TF-IDF   
    tf_idf_score = {}
    tf_idf_score1 = {key: tf_score1[key] * idf_score1.get(key, 0) for key in tf_score1.keys()}
    tf_idf_score2 = {key: tf_score2[key] * idf_score2.get(key, 0) for key in tf_score2.keys()}
    tf_idf_score3 = {key: tf_score3[key] * idf_score3.get(key, 0) for key in tf_score3.keys()}
    
    tf_idf_score.update(tf_idf_score1)
    tf_idf_score.update(tf_idf_score2)
    tf_idf_score.update(tf_idf_score3)

    
    return get_top_n(tf_idf_score, n)

In [21]:
def clean_text(text):
    text = re.sub(r'[)(,'"'=%;’–:"'"><\/—]','',text)
    text = text.replace('- ', '')
    text = re.sub('  ',' ', text) 
    text = text.replace(' .','.')
    text = text.replace(' ?','.')
    text = text.replace(' !','.')
    return text.lower()

In [22]:
def get_top_n(dict_elem, n=0.02):
    res = {}
    for word, score in dict_elem.items():
        if score > n:
            res.update({word: score})
    return sorted(res.items(), key = itemgetter(1), reverse = True)

In [23]:
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

In [24]:
def tf(words):
    tf_score = {}
    for word in words:
        word = word.replace('.','')
        if word not in stop_words:
            if word in tf_score:
                tf_score[word] += 1
            else:
                tf_score[word] = 1
    tf_score.update((x, y/len(words)) for x, y in tf_score.items())
    return tf_score

In [25]:
def idf(words, sentences):
    idf_score = {}
    for word in words:
        word = word.replace('.','')
        if word not in stop_words:
            if word in idf_score:
                idf_score[word] = check_sent(word, sentences)
            else:
                idf_score[word] = 1

    idf_score.update((x, math.log((len(sentences))/y)) for x, y in idf_score.items())
    return idf_score

In [26]:
data['tfidf_not_lem'] = data['abstract'].apply(tf_idf)
data['tfidf_lem'] = data['abstract_lem'].apply(tf_idf)

In [27]:
data['tfidf_lem'] = data['tfidf_lem'].apply(keywords_only)
data['tfidf_not_lem'] = data['tfidf_not_lem'].apply(keywords_only)

Tf_Idf тоже лучше работает с лемматизированным текстом

In [28]:
data['tfidf_lem']

0                             [farm, well, farm animal]
1                                                    []
2                                               [index]
3     [project, external reward motivation, external...
4                                                    []
5     [individual, individual similar personality, i...
6                                           [95 ci, 95]
7         [thinking, symbolic representation, symbolic]
8     [pick love, pick, love, say two people, two pe...
9     [framework, last decade intellectual, decade i...
10                                                   []
11                                           [question]
12    [research, artificial intelligence ai, intelli...
13                                                   []
14                                                   []
15                                                   []
Name: tfidf_lem, dtype: object

In [29]:
data['tfidf_not_lem']

0             []
1             []
2             []
3      [project]
4             []
5             []
6             []
7     [thinking]
8             []
9     [has been]
10            []
11            []
12            []
13            []
14            []
15            []
Name: tfidf_not_lem, dtype: object

### Набираю список шаблонов

In [30]:
def lemm_list(a_list):
    new_list = [lemmatize(word) for word in a_list]
    return new_list

In [31]:
data['rake_not_lem'] = data['rake_not_lem'].apply(lemm_list)

In [32]:
data['tfidf_lem']

0                             [farm, well, farm animal]
1                                                    []
2                                               [index]
3     [project, external reward motivation, external...
4                                                    []
5     [individual, individual similar personality, i...
6                                           [95 ci, 95]
7         [thinking, symbolic representation, symbolic]
8     [pick love, pick, love, say two people, two pe...
9     [framework, last decade intellectual, decade i...
10                                                   []
11                                           [question]
12    [research, artificial intelligence ai, intelli...
13                                                   []
14                                                   []
15                                                   []
Name: tfidf_lem, dtype: object

In [33]:
filters = []
filter_set = set()
for key_list in data['keywords']:

    for key_word in key_list.split(', '):
        pos_tags = []
        tokens = nlp(key_word)
        for token in tokens:
            pos_tags.append(token.pos_)
        pos_filter = '+'.join(pos_tags)
        filters.append(pos_filter)

filter_counts = Counter(filters)

for key, value in filter_counts.items():
    if value > 2:
        filter_set.add(key)

In [34]:
filter_set

{'ADJ+NOUN', 'NOUN', 'NOUN+NOUN', 'NOUN+PROPN', 'PROPN', 'PROPN+PROPN', 'VERB'}

In [35]:
def apply_filter(filters, keywords):
    filtered = []
    for keyword in keywords:
        pos_tags = []
        tokens = nlp(keyword)
        for token in tokens:
            pos_tags.append(token.pos_)
        pos = '+'.join(pos_tags)
        if pos in filters:
            filtered.append(keyword)
    return filtered

In [36]:
data['rake_not_lem_filtered'] = data['rake_not_lem'].apply(lambda x: apply_filter(filter_set, x))

In [37]:
data['TextRank_lem_filtered'] = data['TextRank_lem'].apply(lambda x: apply_filter(filter_set, x))

In [38]:
data['tfidf_lem_filtered'] = data['tfidf_lem'].apply(lambda x: apply_filter(filter_set, x))

### Метрики

In [50]:
def f_score(real, predicted):
    prec = precision(real, predicted)
    rec = recall(real, predicted)
    if prec == 'No keywords were extracted':
        return 'No keywords were extracted'
    elif prec == 0:
        return 'There were no True Positives'
    else:
        f_score = (prec * rec)/(prec + rec)
        return f_score

In [51]:
def precision(real, predicted):
    TP = 0
    for word in real:
        if word in set(predicted):
            TP +=1
    if len(predicted) != 0:
        precision = TP / len(predicted)
        return precision
    else:
        return 'No keywords were extracted'

In [52]:
def recall(real, predicted):
    TP = 0
    for word in real:
        if word in set(predicted):
            TP +=1
    if len(predicted) != 0:
        recall = TP / len(real)
        return recall
    else:
        return 'No keywords were extracted'

In [53]:
data['rake_not_filtered_f_score'] = data.apply(lambda x: f_score(x.keywords_lem,
                                                              x.rake_not_lem), axis=1)
data['TextRank_not_filtered_f_score'] = data.apply(lambda x: f_score(x.keywords_lem,
                                                              x.TextRank_lem), axis=1)
data['tfidf_not_filtered_f_score'] = data.apply(lambda x: f_score(x.keywords_lem,
                                                              x.tfidf_lem), axis=1)
data['rake_not_filtered_precision'] = data.apply(lambda x: precision(x.keywords_lem,
                                                              x.rake_not_lem), axis=1)
data['TextRank_not_filtered_precision'] = data.apply(lambda x: precision(x.keywords_lem,
                                                              x.TextRank_lem), axis=1)
data['tfidf_not_filtered_precision'] = data.apply(lambda x: precision(x.keywords_lem,
                                                              x.tfidf_lem), axis=1)
data['rake_not_filtered_recall'] = data.apply(lambda x: recall(x.keywords_lem,
                                                              x.rake_not_lem), axis=1)
data['TextRank_not_filtered_recall'] = data.apply(lambda x: recall(x.keywords_lem,
                                                              x.TextRank_lem), axis=1)
data['tfidf_not_filtered_recall'] = data.apply(lambda x: recall(x.keywords_lem,
                                                              x.tfidf_lem), axis=1)

In [54]:
data['rake_filtered_f_score'] = data.apply(lambda x: f_score(x.keywords_lem,
                                                              x.rake_not_lem_filtered), axis=1)
data['TextRank_filtered_f_score'] = data.apply(lambda x: f_score(x.keywords_lem,
                                                              x.TextRank_lem_filtered), axis=1)
data['tfidf_filtered_f_score'] = data.apply(lambda x: f_score(x.keywords_lem,
                                                              x.tfidf_lem_filtered), axis=1)
data['rake_filtered_precision'] = data.apply(lambda x: precision(x.keywords_lem,
                                                              x.rake_not_lem_filtered), axis=1)
data['TextRank_filtered_precision'] = data.apply(lambda x: precision(x.keywords_lem,
                                                              x.TextRank_lem_filtered), axis=1)
data['tfidf_filtered_precision'] = data.apply(lambda x: precision(x.keywords_lem,
                                                              x.tfidf_lem_filtered), axis=1)
data['rake_filtered_recall'] = data.apply(lambda x: recall(x.keywords_lem,
                                                              x.rake_not_lem_filtered), axis=1)
data['TextRank_filtered_recall'] = data.apply(lambda x: recall(x.keywords_lem,
                                                              x.TextRank_lem_filtered), axis=1)
data['tfidf_filtered_recall'] = data.apply(lambda x: recall(x.keywords_lem,
                                                              x.tfidf_lem_filtered), axis=1)

Добавление фильтра в некоторых случаях улучшает f-score.

In [55]:
data[['rake_not_filtered_f_score', 'rake_filtered_f_score']]

,rake_not_filtered_f_score,rake_filtered_f_score
0,0.166667,0.181818
1,0.214286,0.214286
2,0.0909091,0.125
3,0.0833333,0.0833333
4,0.111111,0.111111
5,0.117647,0.133333
6,0.1875,0.214286
7,0.277778,0.235294
8,0.181818,0.2
9,0.125,0.125


In [56]:
data[['TextRank_not_filtered_f_score', 'TextRank_filtered_f_score']]

,TextRank_not_filtered_f_score,TextRank_filtered_f_score
0,There were no True Positives,There were no True Positives
1,There were no True Positives,There were no True Positives
2,0.0434783,0.0625
3,0.0526316,0.0666667
4,0.0526316,0.0625
5,There were no True Positives,There were no True Positives
6,0.0454545,0.0555556
7,0.0714286,0.0769231
8,0.0769231,0.0909091
9,0.133333,0.153846


In [57]:
data[['tfidf_not_filtered_f_score', 'tfidf_filtered_f_score']]

,tfidf_not_filtered_f_score,tfidf_filtered_f_score
0,0.111111,0.125
1,No keywords were extracted,No keywords were extracted
2,There were no True Positives,There were no True Positives
3,0.00913242,0.0289855
4,No keywords were extracted,No keywords were extracted
5,There were no True Positives,There were no True Positives
6,There were no True Positives,No keywords were extracted
7,There were no True Positives,There were no True Positives
8,0.0145631,0.0526316
9,0.00529101,0.0140845


Я заметила, что предсказанные алгоритмами ключевые слова часто близки к эталону, но не совпадает деление биграмм/триграмм. Для наглядности и для того, чтобы сравнить методы между собой, решила посмотреть какие отдельные слова, вообще, сопадают в эталонных и предсказанных.

In [58]:
def similarity(real, predicted):
    real_string = ' '.join(real)
    pred_string = ' '.join(predicted)
    return set(real_string.split()) & set(pred_string.split())

In [59]:
data['TextRank_sims'] =  data.apply(lambda x: similarity(x.keywords_lem,
                                                              x.TextRank_lem), axis=1)
data['rake_sims'] =  data.apply(lambda x: similarity(x.keywords_lem,
                                                              x.rake_not_lem), axis=1)
data['tfidf_sims'] =  data.apply(lambda x: similarity(x.keywords_lem,
                                                              x.tfidf_lem), axis=1)

In [60]:
data['TextRank_sims']

0                  {value, environmental, belief, farm}
1                          {social, sociology, pattern}
2                          {environmental, social, adb}
3                            {efficacy, science, youth}
4         {consumer, green, empowerment, intention, ad}
5               {identity, facial, person, correlation}
6        {traumatic, parent, post, ptsd, factor, child}
7     {intelligence, cognition, emotion, ai, computi...
8                              {experiment, true, love}
9                 {humility, measurement, intellectual}
10                    {end, risk, motivation, decision}
11         {intelligence, control, ai, artificial, agi}
12                                         {ai, animal}
13      {consumer, innovation, resistance, self, brand}
14                               {pandemic, health, ai}
15                                    {ci, method, fsc}
Name: TextRank_sims, dtype: object

In [61]:
data['rake_sims']

0          {value, environmental, farm, belief, animal}
1                          {social, sociology, pattern}
2                                                 {adb}
3                         {efficacy, science, interest}
4                       {purchase, intention, consumer}
5         {perception, face, facial, feature, identity}
6                                 {ptsd, parent, child}
7     {intelligence, emotion, ai, think, consciousness}
8                                          {true, love}
9                              {humility, intellectual}
10              {risky, end, risk, effect, decision, -}
11                                                 {ai}
12                                         {ai, animal}
13                                  {brand, connection}
14                             {pandemic, covid-19, ai}
15                                    {ci, method, fsc}
Name: rake_sims, dtype: object

In [62]:
data['tfidf_sims']

0                                  {farm, well, animal}
1                                                    {}
2                                                    {}
3     {efficacy, citizen, political, education, scie...
4                                                    {}
5                                                    {}
6                                                    {}
7                                            {thinking}
8           {concept, ordinary, true, experiment, love}
9     {definition, humility, measurement, intellectual}
10                                                   {}
11                                                   {}
12    {conservation, intelligence, ai, artificial, a...
13                                                   {}
14                                                   {}
15                                                   {}
Name: tfidf_sims, dtype: object

## Выводы

В целом автоматические методы плохо справились, думаю, что в первую очередь это связано с тем, что я использовала очень маленькие тексты + оригинальные ключевые слова относились к статье в целом, а не только к аннотации. Алгоритмы воспринимают абревиатуры и их расшифровки как отдельные слова, а живые люди, конечно, нет. Многие важные детали в текстах не являются частотными - например, информация о методах и типах экспериментов.
Если сравнить методы между собой, то сильно хуже других справился Tf-Idf, TextRank и Rake ошибались в разных местах.